In [2]:
import requests
import folium
import random
import ipywidgets as widgets
from IPython.display import display, clear_output

# BUSCA DE PAÍS

In [ ]:
# Função para tratar o clique no botão
def on_search_click(b):
    clear_output()  # Limpar saída anterior
    display(widgets.HBox([text_input, search_button]))
    search_term = text_input.value.strip()
    if search_term:
        fetch_country_data(search_term)
    else:
        print("Please, enter a country name")

# Função para buscar e exibir dados do país
def fetch_country_data(country_name):
    url = f"https://restcountries.com/v3.1/name/{country_name}?fields=name,latlng,flags,region,subregion,population,capital,demonyms"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Gera um erro para status HTTP >= 400
        country_data = response.json()
        
        if not country_data:
            print(f"Empty results for '{country_name}'.")
            return
        
        # Extraindo os dados do país
        data = country_data[0]
        lat, lng = data.get('latlng', [None, None])
        flag_url = data.get('flags', {}).get('png', 'N/A')
        region = data.get('region', 'Unkwown')
        subregion = data.get('subregion', 'Unkwown')
        population = data.get('population', 'N/A')
        capital = ', '.join(data.get('capital', ['N/A']))
        demonyms = data.get("demonyms", {}).get("eng", {}).get("m", "N/A")
        
        if lat is None or lng is None:
            print(f"Coordenadas não disponíveis para '{country_name}'.")
            return
        
        # Criar o mapa
        country_map = folium.Map(location=[lat, lng], zoom_start=4)
        
        # Conteúdo do popup no mapa
        popup_content = f"""
        <h3>{data['name']['common']}</h3>
        <img src="{flag_url}" alt="Flag of {country_name}" style="width:100px; height:auto;">
        <p><strong>Region:</strong> {region}</p>
        <p><strong>Subregion:</strong> {subregion}</p>
        <p><strong>Population:</strong> {population:,}</p>
        <p><strong>Capital:</strong> {capital}</p>
        <p><strong>Demonym:</strong> {demonyms}</p>
        """
        folium.Marker(location=[lat, lng], popup=folium.Popup(popup_content, max_width=300), tooltip=f"{data['name']['common']}").add_to(country_map)
        
        # Exibir o mapa
        display(country_map)
    
    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar dados do país: {e}")
    except KeyError as e:
        print(f"Erro ao processar dados do país: Campo faltando ({e}).")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

# Criar campo de texto e botão
text_input = widgets.Text(description='Search:', placeholder='Enter a country...', layout=widgets.Layout(width='300px'))
search_button = widgets.Button( description='Search', button_style='success', tooltip='Click for search', icon='search')

# Conectar o botão à função
search_button.on_click(on_search_click)

# Exibir os widgets
display(widgets.HBox([text_input, search_button]))

# Funções comuns ao jogos de bandeira e capital

In [4]:

# Função para obter dados de países
def get_country_data(country):
    try:
        name = country["name"]["common"]
    except (KeyError, TypeError):
        name = "Desconhecido"

    try:
        capital = country.get("capital", ["Desconhecida"])[0]
    except (IndexError, TypeError):
        capital = "Desconhecida"

    try:
        region = country.get("region", "Desconhecida")
    except (KeyError, TypeError):
        region = "Desconhecida"

    try:
        flag = country["flags"]["png"]
    except (KeyError, TypeError):
        flag = "Desconhecida"

    return {
        "name": name,
        "capital": capital,
        "region": region,
        "flag": flag
    }


def fetch_country_data():
    url = "https://restcountries.com/v3.1/all?fields=name,flags,capital,region"
    response = requests.get(url)
    if response.status_code == 200:
        countries = response.json()
        return  [
                get_country_data(country)
                for country in countries
            ]
    else:
        print("Erro ao acessar a API.")
        return []


#### JOGO DE ADIVINHAR PAÍSES COM BASE NA BANDEIRA

In [ ]:
# Função para exibir a rodada
def play_round():
    global current_round, score
    
    if current_round < 5:
        # Selecionar o país da rodada
        country = game_countries[current_round]
        
        # Mostrar a dica (bandeira)
        print(f"Round {current_round + 1}/5: What is the country of this flag?")
        display(widgets.Image(value=requests.get(country["flag"]).content))
        
        # Criar opções de resposta
        options = random.sample([c["name"] for c in countries], 4)
        if country["name"] not in options:
            options[0] = country["name"]  # Garantir que a resposta correta está nas opções
        random.shuffle(options)  # Embaralhar opções
        
        buttons = widgets.ToggleButtons(
            options=options,
            description="Choose:",
            value=None
        )
        
        # Botão para confirmar a resposta
        confirm_button = widgets.Button(description="Confirm", button_style="info")

        # "Next" fica desabilitado
        next_button = widgets.Button(description="Next", button_style="success")
        next_button.disabled = True
        
        def check_answer(b):
            global current_round, score
            confirm_button.disabled = True
            if buttons.value == country["name"]:
                print("✅ Correct answer!")
                score += 1
            else:
                print(f"❌ Wrong answer! The correct country is: {country['name']}")
                      
            next_button.disabled = False

        def go_to_next_round(b):
            global current_round
            current_round += 1
            clear_output()
            play_round()

        confirm_button.on_click(check_answer)
        next_button.on_click(go_to_next_round)
        
        display(buttons, widgets.HBox((confirm_button, next_button)))
    else:
        # Fim do jogo
        print(f"🏁 End of game! Your final score is {score}/5.")
        if score == 5:
            print("🎉 Excelent! You got all right!")
        elif score >= 3:
            print("👏 Good job! You did well!")
        else:
            print("💡 Keep practicing, you'll get better!")
        
        play_again_button = widgets.Button(description="Play Again")
        play_again_button.on_click(play_again)
        display(play_again_button)

def play_again(_):
    global current_round, score, game_countries
    current_round = 0
    score = 0
    game_countries = random.sample(countries, 5)
    clear_output()
    play_round()


# Dados de países
countries = fetch_country_data()
if len(countries) < 5:
    raise ValueError("There's no data enough. Check the API")

# Selecionar 5 países aleatórios para o jogo
game_countries = random.sample(countries, 5)
score = 0  # Pontuação
current_round = 0  # Rodada atual

# Iniciar o jogo
print("Welcome to flag quiz!")
play_round()


#### JOGO DE ADIVINHAR PAÍSES COM BASE NA CAPITAL

In [ ]:
# Função para exibir a rodada
def play_round():
    global current_round, score
    
    if current_round < 5:
        # Selecionar o país da rodada
        country = game_countries[current_round]
        
        # Mostrar a dica (bandeira)
        print(f"Round {current_round + 1}/5: Which country the capital '{country['capital']}' belongs to?")
        
        # Criar opções de resposta
        options = random.sample([c["name"] for c in countries], 4)
        if country["name"] not in options:
            options[0] = country["name"]  # Garantir que a resposta correta está nas opções
        random.shuffle(options)  # Embaralhar opções
        
        buttons = widgets.ToggleButtons(
            options=options,
            description="Choose:",
            value=None
        )
        
        # Botão para confirmar a resposta
        confirm_button = widgets.Button(description="Confirm", button_style="info")

        # Inicialmente, o botão "Next" está desabilitado
        next_button = widgets.Button(description="Next", button_style="success")
        next_button.disabled = True
        
        def check_answer(b):
            global current_round, score
            confirm_button.disabled = True
            if buttons.value == country["name"]:
                print("✅ Correct answer!")
                score += 1
            else:
                print(f"❌ Wrong answer! The correct country is: {country['name']}")
                      
            next_button.disabled = False

        def go_to_next_round(b):
            global current_round
            current_round += 1
            clear_output()
            play_round()

        confirm_button.on_click(check_answer)
        next_button.on_click(go_to_next_round)
        
        display(buttons, widgets.HBox((confirm_button, next_button)))
    else:
        # Fim do jogo
        print(f"🏁 End of game! Your final score is {score}/5.")
        if score == 5:
            print("🎉 Excelent! You got all right!")
        elif score >= 3:
            print("👏 Good job! You did well!")
        else:
            print("💡 Keep practicing, you'll get better!")
        
        play_again_button = widgets.Button(description="Play Again")
        play_again_button.on_click(play_again)
        display(play_again_button)

def play_again(_):
    global current_round, score, game_countries
    current_round = 0
    score = 0
    game_countries = random.sample(countries, 5)
    clear_output()
    play_round()

# Dados de países
countries = fetch_country_data()
if len(countries) < 5:
    raise ValueError("There's no data enough. Check the API")

# Selecionar 5 países aleatórios para o jogo
game_countries = random.sample(countries, 5)
score = 0  # Pontuação
current_round = 0  # Rodada atual

# Iniciar o jogo
print("Welcome to flag quiz trough capitals")
play_round()
